In [1]:
# This tool will not ensure that pixels and boundaries align
# That is the job of the user
import pandas
import numpy
from osgeo import gdal, gdalconst
from osgeo import ogr
import tempfile
import os
import sys
sys.path.append('../Modules')
import zonal_stats_RasterInput
import resample_raster
#from rasterstats import zonal_stats
import matplotlib.pyplot as plt
%matplotlib inline

#Path to intersected Subcatch/FUs shapefile
pathToShapefile = "D:/MW/spatial_RC7/ExportedCatchments-FUs Intersection.shp"
#pathToShapefile = "D:/MW/spatial_RC7/mw_intscfu_clean.shp"
#pathToShapefile = "D:/MW/spatial_RC7/badFUOnly.shp"

uniqueIDField = "UNQID"
catchmentField = "IntSCs"
FUField = "IntFUs"

FUsToInclude = ["Grazing Open","Grazing Forested","Forestry","Conservation"]

#Path to template raster (probably subcatchments or DEM)
rasterTemplateFile = "D:/MW/spatial_RC7/CreatedSubcats.asc"

#Output intersected Subcat/FU raster
#Will get a CSV of same name written 'mapping' each FU to a unique ID (the unique ID will be the raster values)
rasterOutputFile = "D:/MW/spatial_RC7/MW_USLE_IntSubCatFu.tif"

#Path to K raster
rasterKFile = "D:/MW/Ram_USLE/mw_k_fact.asc"

#Path to LS raster
rasterLSFile = "D:/MW/Ram_USLE/mw_ls_fact.asc"

#Path to Fines raster
rasterFinesFile = "D:/MW/Ram_USLE/mw_cs_surf.asc"

#Path to Scald raster
rasterScaldFile = "D:/MW/spatial_RC7/CreatedSubcats.asc"

#Directory of CFactor rasters
cFactorDir = "D:/MW/spatial_RC7"

#declare a couple of variables
IntSCFUStr = "IntSCFU"
MeanKLSCStr = "Mean_KLSC"
MeanKLSCFinesStr = "Mean_KLSC_Fines"
MeanKStr = "Mean_K"
MeanFinesStr = "Mean_Fines"
MeanCfactStr = "Mean_CFact"
MeanLSStr = "Mean_LS"
MeanScaldStr = "Mean_Scald"
AreaStr = "Area"
TimeStampStr = "Time_Stamp"
CellCountStr = "Cell_Count"

#Directory and filename constants for USLE parameterisation
USLE_DIR_KLSC = "KLSC";
USLE_DIR_KLSCFINE = "KLSC_Fines";
USLE_DIR_KLSC_CFACT = "Cfact";
USLE_FILE_KLSC = "USLE_KLSC_Total";
USLE_FILE_KLSCFINES = "USLE_KLSC_FinePerc";
USLE_FILE_KLS_CFACT = "CFactor";

rasterResultsDF = pandas.DataFrame(columns=[IntSCFUStr,TimeStampStr,MeanKLSCStr,MeanKLSCFinesStr,MeanKStr,MeanCfactStr,MeanFinesStr,MeanLSStr,MeanScaldStr,AreaStr,CellCountStr])
#rasterResultsDF

In [2]:
#load the template raster
templateRas = gdal.Open(rasterTemplateFile)
template_GeoTrans = templateRas.GetGeoTransform()
template_GeoTrans

(1677868.63051, 30.0, 0.0, -2257960.4690071, 0.0, -30.0)

In [3]:
#Load K factor
kFactRas = gdal.Open(rasterKFile)
kfact_GeoTrans = kFactRas.GetGeoTransform()
kfact_GeoTrans

(1672918.63051, 30.0, 0.0, -2251120.4690071, 0.0, -30.0)

In [4]:
kFactRas.GetRasterBand(1).GetNoDataValue()

-9999.0

In [5]:
if (kfact_GeoTrans == template_GeoTrans):
    print("Kfactor raster resolution info matches template, excellent")
else:
    print("Kfactor raster resolution info didn't match template, attempted a resample")
    kFactRas = resample_raster.resample_raster_to_match_template(templateRas, kFactRas, os.path.join(tempfile.gettempdir(), "resampledKFact.tif"))
    #kFactRas = resample_raster_to_match_template(templateRas, kFactRas, "D:/aa/tester.tif")
    kfact_GeoTrans = kFactRas.GetGeoTransform()

Kfactor raster resolution info didn't match template, attempted a resample


In [6]:
if (kfact_GeoTrans == template_GeoTrans):
    print("Kfactor raster resolution info now matches template, excellent")
    kFactRas = None
    kFactRas = gdal.Open(os.path.join(tempfile.gettempdir(), "resampledKFact.tif"))
    kfact_GeoTrans = kFactRas.GetGeoTransform()
else:
    print("Kfactor raster resolution still not the same as the template, you shouldn't proceed")
#zone_ds = None
#print("rows: " + str(kFactRas.RasterYSize) + " cols: " + str(kFactRas.RasterXSize))

Kfactor raster resolution info now matches template, excellent


In [7]:
#kFactRas.GetRasterBand(1).GetNoDataValue()
#templateRas.GetRasterBand(1).GetNoDataValue()

# The line below would get the kfactor raster written to disk, but it may not be necessary
#kFactRas = None

#this stuf to draw up a sample of KFactor, have to chop off heaps of edges so that the range in the colorbar doesn't consider -9999
#kFactRasArray = kFactRas.ReadAsArray()
#array_chop = kFactRasArray[3500:-3500,1500:-1500]  # chop off the outer specified pixels
#plt.imshow(array_chop, interpolation='nearest')
#plt.colorbar()
#plt.title('K Factor')
#plt.show()

kfact_GeoTrans

(1677868.63051, 30.0, 0.0, -2257960.4690071, 0.0, -30.0)

In [8]:
#kFactRas.GetRasterBand(1).GetNoDataValue()
kFactRas.RasterXSize

3823

In [9]:
kFactRas.RasterYSize

8417

In [10]:
#Open shapefile
basePolys = ogr.Open(pathToShapefile)
type(basePolys)
nlay = basePolys.GetLayerCount()
lyr = basePolys.GetLayer(0)
ext = lyr.GetExtent()
#nlay
#ext
lyr.GetFeatureCount()
#plt.imshow(lyr)

1286

In [11]:
featCount = 0
for feat in lyr:
    ##print("Doing something")
    #print(feat.GetFieldAsString(catchmentField) + " and FU " + feat.GetFieldAsString(FUField))
    featCount += 1
    if (featCount == 20):
        break

In [12]:
#FUsToInclude = ["Grazing Open","Grazing Forested","Forestry"]

FUsAsSQL = ""
count = 0
for thisFU in FUsToInclude:
    if(count == 0):
        FUsAsSQL = "'" + thisFU + "'"
    else:
        FUsAsSQL += ",'" + thisFU + "'"
    count += 1

FUsAsSQL

"'Grazing Open','Grazing Forested','Forestry','Conservation'"

In [13]:
lyr.SetAttributeFilter( FUField + " IN (" + FUsAsSQL + ")")
lyr.GetFeatureCount()

644

In [14]:
##help(templateRas)
geo_transform = templateRas.GetGeoTransform()
x_min = geo_transform[0]
y_max = geo_transform[3]
x_max = x_min + geo_transform[1] * templateRas.RasterXSize
y_min = y_max + geo_transform[5] * templateRas.RasterYSize
x_count = templateRas.RasterXSize
y_count = templateRas.RasterYSize
pixel_width = geo_transform[1]
pixel_height = geo_transform[5]
#y_min

In [15]:
zone_ds = gdal.GetDriverByName("GTiff").Create(rasterOutputFile, x_count, y_count, 1, gdal.GDT_Int32)
##Doing MEM did not yield the expected unique vals
##zone_ds = gdal.GetDriverByName("MEM").Create("", x_count, y_count, 1, gdal.GDT_Byte)
# This was shifting the resulting raster northwards in our Albers stuff
#zone_ds.SetGeoTransform((x_min, pixel_width, 0, y_min, 0, pixel_width))
zone_ds.SetGeoTransform((x_min, pixel_width, 0, y_max, 0, pixel_height))
band = zone_ds.GetRasterBand(1)
NoData_value = -9999
band.SetNoDataValue(NoData_value)
band.FlushCache()
gdal.RasterizeLayer(zone_ds, [1], lyr, options=["ATTRIBUTE=" + uniqueIDField])

0

In [16]:
zone_ds.RasterXSize

3823

In [17]:
zone_ds.RasterYSize

8417

In [18]:
##This bit essential to get the raster file written - but could we use it before writing out (and then having to read back in?)
##Hopefully not needed if MEM type
##zone_ds = None
type(zone_ds)

osgeo.gdal.Dataset

In [19]:
zone_ds.GetGeoTransform()

(1677868.63051, 30.0, 0.0, -2257960.4690071, 0.0, -30.0)

In [20]:
#zonesRasterArray = zone_ds.ReadAsArray()
#uniq = numpy.unique(zonesRasterArray)
##print(uniq)


In [21]:
#type(lyr)
#stats = zonal_stats(lyr, kFactRas)

In [22]:
#K factor zonal stats
Kfact_stats = zonal_stats_RasterInput.zonal_stats(zone_ds, kFactRas)
# Will get 'converting a masked element to nan' error for tiny polygons

DataRaster NoDataValue: -9999.0
ZoneRaster NoDataValue: -9999.0
Doing zonal stats on 644 unique zones
Ignoring value: -9999
Result for feat ZID: 0 Mean: 0.0495116523872 Count: 19682 Min: 0.0256 Max: 0.1045 Sum: 974.488
Result for feat ZID: 1 Mean: 0.0515408248334 Count: 37759 Min: 0.0256 Max: 0.0956 Sum: 1946.13
Result for feat ZID: 2 Mean: 0.0552034508395 Count: 51070 Min: 0.0335 Max: 0.1045 Sum: 2819.24


KeyboardInterrupt: 

In [23]:
kFactRas = None
#lsFactRas = None
#FinesRas = None

index = 0
while index < len(Kfact_stats):
    if(Kfact_stats[index][uniqueIDField] < 25):
        print(str(Kfact_stats[index][uniqueIDField]) + " KFactor Mean: " + str(Kfact_stats[index]['mean']) + " Count: " + str(Kfact_stats[index]['count']))
        #print(str(LSfact_stats[index][uniqueIDField]) + " LS Mean: " + str(LSfact_stats[index]['mean']) + " Count: " + str(LSfact_stats[index]['count']))
        #print(str(Fines_stats[index][uniqueIDField]) + " Fines Mean: " + str(Fines_stats[index]['mean']) + " Count: " + str(Fines_stats[index]['count']))
        break
    
    index += 1

NameError: name 'Kfact_stats' is not defined